<h1><font color=darkred size=6><center> $\fbox{Préparation des données}$ </center></h1>

*Ce jupyter notebook ne contient pas notre projet.*<br>
*Le contenu de notre projet se trouve dans le fichier intitulé **NOM.ipynb**.*<br>
<br>
Ce notebook contient le travail préalable que nous avons effectué pour la **préparation de nos données** comme le chargement, le nettoyage, le reformatage et l'organisation de nos données avant de les utiliser pour le projet de statistique et apprentissage pour la prévision.<br>
Nos données proviennent de :
- <a href="https://www.data.gouv.fr/fr/datasets/historique-des-donnees-de-validation-sur-le-reseau-de-surface-2015-2021/">Historique des données de validation (2015-2021)</a>
- <a href="https://www.jour-ferie.info/dates-des-fetes-et-jours-feries-en-2022/">Jours fériés et jours de fête</a>
- <a href="https://www.cestlagreve.fr/">Jours de grève</a>
- <a href="https://vacances-scolaires.education/annee-2021-2022.php">Vacances scolaires</a>
- <a href="https://fr.wikipedia.org/wiki/Confinements_li%C3%A9s_%C3%A0_la_pand%C3%A9mie_de_Covid19_en_France#:~:text=du%2017%20mars%20au%2011,non%20inclus%2C%20soit%2028%20jours">Confinements</a>
- <a href="https://www.vie-publique.fr/en-bref/277391-covid-19-couvre-feu-18-heures-partir-du-16-janvier">Couvre-feu</a>



⚠️ Il n'y a pas besoin de run ce notebook, nous l'avons déjà fait ! ⚠️<br>

<h3><font color=darkred>Importation des librairies et configuration de l'affichage</h3>

In [1]:
import os
import csv
import pandas as pd
import warnings
from datetime import datetime
import time

start_time = time.time()

In [2]:
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.max_columns = None

<h3><font color=darkred>Chargement et reformatage des données</h3>

Nous allons reformater nos données brutes de telle sorte à ce qu'elles soient disponibles par année et non plus par semestre (en les concatenant).<br>
<br>
Lors de ce premier passage, on en profitera pour :
- Regrouper le nombre de validation par jour et par arrêt.
- Homogénéiser le type de données de la colonne 'NB_VALD' pour avoir uniquement des entiers (remplacer les strings "Moins de 5" par 5)

In [3]:
liste_annee = ['2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [4]:
for annee in liste_annee :
    
    if annee == '2015' :
        df1 = pd.read_csv('Frequentation/'+annee+'_S1_NB_FER.csv', sep=';')
        df2 = pd.read_csv('Frequentation/'+annee+'_S2_NB_FER.csv', sep=';')
    else :
        df1 = pd.read_csv('Frequentation/'+annee+'_S1_NB_FER.txt', sep='\t')
        df2 = pd.read_csv('Frequentation/'+annee+'_S2_NB_FER.txt', sep='\t')
        
    df1.loc[df1['NB_VALD'] == "Moins de 5"] = 5

    df1 = df1.astype({'NB_VALD': 'int32'})
    df1 = df1.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].agg('sum')
    df1.to_csv(annee+"_S1.csv")
    
    df2.loc[df2['NB_VALD'] == "Moins de 5"] = 5

    df2 = df2.astype({'NB_VALD': 'int32'})
    df2 = df2.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].agg('sum')
    df2.to_csv(annee+"_S2.csv")
    
    df1 = pd.read_csv(annee+"_S1.csv")
    df2 = pd.read_csv(annee+"_S2.csv")
    df = pd.concat([df1,df2], ignore_index=True)
    df.to_csv(annee+".csv")
    
    os.remove(annee+"_S1.csv") 
    os.remove(annee+"_S2.csv")
    
    print(annee, " ok")

2015  ok
2016  ok
2017  ok
2018  ok
2019  ok
2020  ok
2021  ok


In [5]:
df2.head()

,JOUR,LIBELLE_ARRET,NB_VALD
0,01/07/2021,ABBESSES,3207
1,01/07/2021,ABLON,1574
2,01/07/2021,ACHERES-GRAND-CORMIER,65
3,01/07/2021,ACHERES-VILLE,4090
4,01/07/2021,AEROPORT CHARLES DE GAULLE 1,4578


<h3><font color=darkred>Concaténation des bases de données</h3>

Après avoir regroupé nos données par année, nous supprimerons également la colonne "Unnamed: 0" de tous les DataFrames puis nous allons les concatener de sorte à avoir un seul et même DataFrame, appelé **df_final**, contenant les informations de toutes les années (de 2015 à 2021).<br>
Les fichiers csv intermédiaires contenants les informations par années seront supprimées.

In [6]:
df_finale = pd.read_csv("2015.csv", sep=",").drop("Unnamed: 0", axis=1)
os.remove("2015.csv")

for annee in liste_annee[1:] :
    df_temp = pd.read_csv(annee+".csv", sep=",").drop("Unnamed: 0", axis=1)
    df_finale = pd.concat([df_finale, df_temp], ignore_index=True)
    os.remove(annee+".csv")

In [7]:
df_finale.head()

,JOUR,LIBELLE_ARRET,NB_VALD
0,5,5,283355
1,01/01/2015,ABBESSES,750
2,01/01/2015,ABLON,108
3,01/01/2015,ACHERES-GRAND-CORMIER,8
4,01/01/2015,ACHERES-VILLE,482


<h3><font color=darkred>Stations Ligne 1</h3>

Dans le cadre de notre projet, nous avons choisi de nous intéresser uniquement aux stations désservies par la ligne 1.<br>
<br>
Pour ce faire, ne allons conserver que les lignes où la valeur de la colonne "LIBELLE_ARRET" correspond à une station de la ligne 1.

In [8]:
liste_ligne_1 = ["LA DEFENSE-GRANDE ARCHE", "ESPLANADE DE LA DEFENSE", "PONT DE NEUILLY (AVENUE DE MADRID)",
                 "LES SABLONS (JARDIN D'ACCLIMATATION)", "PORTE MAILLOT","ARGENTINE", "CHARLES DE GAULLE ETOILE", 
                 "GEORGE V", "FRANKLIN-D.ROOSEVELT", "CHAMPS-ELYSEES-CLEMENCEAU", "CONCORDE", "TUILERIES", 
                 "PALAIS ROYAL-MUSEE DU LOUVRE", "LOUVRE-RIVOLI", "CHATELET", "HOTEL DE VILLE", 
                 "SAINT-PAUL (LE MARAIS)", "BASTILLE", "GARE DE LYON", "REUILLY-DIDEROT", "NATION", 
                 "PORTE DE VINCENNES", "SAINT-MANDE-TOURELLE", "BERAULT", "CHATEAU DE VINCENNES"]

In [9]:
df_finale = df_finale[df_finale["LIBELLE_ARRET"].isin(liste_ligne_1)]

<h3><font color=darkred>Formatage de la colonne "JOUR"</h3>

Nous allons extraire les parties de chaque valeur de la colonne "JOUR" et les combiner en une nouvelle chaîne de caractères au format "AAAA-MM-JJ".<br>
Puis, nous allons convertir ces chaînes de caractères en objets datetime.

In [10]:
df_finale["JOUR"] = df_finale["JOUR"].str.slice(start=6, stop=10)+"-"+df_finale["JOUR"].str.slice(start=3, stop=5)+"-"+df_finale["JOUR"].str.slice(start=0, stop=2)
df_finale["JOUR"] = pd.to_datetime(df_finale["JOUR"])

In [11]:
os.makedirs("Bases", exist_ok=True)
df_finale.to_csv("Bases/Base_Finale.csv")

<h3><font color=darkred>Rajout de datasets auxilliaires</h3>

À notre dataset principal nous avons eu l'idée d'y integrer des datasets auxilières afin d'y ajouter des informations supplémentaires qui peuvent expliquer certaines fluctuations de l'affluence des métros parisiens.<br>
<br>
Nous avons choisi plusieurs datasets qui listent depuis le 1er janvier 2015 :
- Les jours fériés
- Les jours de grèves
- Les jours de fêtes
- Les vacances scolaires
- Les périodes de confinement
- Les périodes de couvre-feu


Pour ce faire, nous allons procéder de manière similaire pour tous les datasets auxilliaires : <br>
Après le chargement des données, nous allons d'abord extraire les parties de chaque valeur de la colonne "JOUR" des DataFrames (jours_feries, jours_greves, jours_fete, vacances, confinements et couvres_feu) et les combiner en une nouvelle chaîne de caractères au format "AAAA-MM-JJ". Puis, nous allons convertir ces chaînes de caractères en objets datetime.<br>
Enfin, nous allons concaténer chacun des DataFrames avec le DataFrame principal.

<h4><font color=darkred>Rajout des Jours Fériés</h4>

In [12]:
jours_feries = pd.read_csv("Caracteristiques_Jours/Jours_Feries.csv", sep=";")[["JOUR", "LIBELLE_JOUR_FERIE"]]

In [13]:
jours_feries.head(1)

,JOUR,LIBELLE_JOUR_FERIE
0,01/01/2015,Jour de l’an


In [14]:
jours_feries["JOUR"] = jours_feries["JOUR"].str.slice(start=6, stop=10)+"-"+jours_feries["JOUR"].str.slice(start=3, stop=5)+"-"+jours_feries["JOUR"].str.slice(start=0, stop=2)
jours_feries["JOUR"] = pd.to_datetime(jours_feries["JOUR"])

In [15]:
df_finale = df_finale.join(jours_feries.set_index('JOUR'), on = "JOUR")

In [16]:
df_finale.tail()

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE
1802375,2021-12-31,PORTE MAILLOT,3897,NaN
1802394,2021-12-31,REUILLY-DIDEROT,6777,NaN
1802429,2021-12-31,SAINT-MANDE-TOURELLE,5087,NaN
1802442,2021-12-31,SAINT-PAUL (LE MARAIS),5061,NaN
1802495,2021-12-31,TUILERIES,361,NaN


<h4><font color=darkred>Rajout des Jours de Grèves</h4>

In [17]:
jours_greves = pd.read_csv("Caracteristiques_Jours/Greves.csv", sep=";")[["JOUR", "MODALITE_GREVE"]]

In [18]:
jours_greves.head(1)

,JOUR,MODALITE_GREVE
0,01/01/2019,Fort


In [19]:
jours_greves["JOUR"] = jours_greves["JOUR"].str.slice(start=6, stop=10)+"-"+jours_greves["JOUR"].str.slice(start=3, stop=5)+"-"+jours_greves["JOUR"].str.slice(start=0, stop=2)
jours_greves["JOUR"] = pd.to_datetime(jours_greves["JOUR"])

In [20]:
df_finale = df_finale.join(jours_greves.set_index('JOUR'), on = "JOUR")

In [21]:
df_finale.head(1)

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE
18,2015-01-01,ARGENTINE,874,Jour de l’an,NaN


<h4><font color=darkred>Rajout des Jours de Fêtes</h4>

In [22]:
jours_fetes = pd.read_csv("Caracteristiques_Jours/Jours_Fetes.csv", sep=";")[["JOUR", "LIBELLE_FETE"]]

In [23]:
jours_fetes.head(1)

,JOUR,LIBELLE_FETE
0,06/01/2015,Epiphanie


In [24]:
jours_fetes["JOUR"] = jours_fetes["JOUR"].str.slice(start=6, stop=10)+"-"+jours_fetes["JOUR"].str.slice(start=3, stop=5)+"-"+jours_fetes["JOUR"].str.slice(start=0, stop=2)
jours_fetes["JOUR"] = pd.to_datetime(jours_fetes["JOUR"])

In [25]:
df_finale = df_finale.join(jours_fetes.set_index('JOUR'), on = "JOUR")

In [26]:
df_finale.head(1)

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE
18,2015-01-01,ARGENTINE,874,Jour de l’an,NaN,NaN


<h4><font color=darkred>Rajout des Vacances</h4>

In [27]:
vacances = pd.read_csv("Caracteristiques_Jours/Vacances.csv", sep=";")[["JOUR","LIBELLE_VACANCE"]]

In [28]:
vacances.head(1)

,JOUR,LIBELLE_VACANCE
0,18/10/2014,Toussaint


In [29]:
vacances["JOUR"] = vacances["JOUR"].str.slice(start=6, stop=10)+"-"+vacances["JOUR"].str.slice(start=3, stop=5)+"-"+vacances["JOUR"].str.slice(start=0, stop=2)
vacances["JOUR"] = pd.to_datetime(vacances["JOUR"])

In [30]:
df_finale = df_finale.join(vacances.set_index('JOUR'), on = "JOUR")

In [31]:
df_finale.head(1)

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE,LIBELLE_VACANCE
18,2015-01-01,ARGENTINE,874,Jour de l’an,NaN,NaN,Noël


<h4><font color=darkred>Rajout des Confinements</h4>

In [32]:
confinements = pd.read_csv("Caracteristiques_Jours/Confinements.csv", sep=";")[["JOUR", "LIBELLE_CONFINEMENT"]]

In [33]:
confinements.head(1)

,JOUR,LIBELLE_CONFINEMENT
0,17/03/2020,Premier Confinement


In [34]:
confinements["JOUR"] = confinements["JOUR"].str.slice(start=6, stop=10)+"-"+confinements["JOUR"].str.slice(start=3, stop=5)+"-"+confinements["JOUR"].str.slice(start=0, stop=2)
confinements["JOUR"] = pd.to_datetime(confinements["JOUR"])

In [35]:
df_finale = df_finale.join(confinements.set_index('JOUR'), on = "JOUR")

In [36]:
df_finale.head(1)

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE,LIBELLE_VACANCE,LIBELLE_CONFINEMENT
18,2015-01-01,ARGENTINE,874,Jour de l’an,NaN,NaN,Noël,NaN


<h4><font color=darkred>Rajout des Couvres-Feu</h4>

In [37]:
couvres_feu = pd.read_csv("Caracteristiques_Jours/Couvres_Feu.csv", sep=";")[["JOUR", "LIBELLE_COUVRE_FEU"]]

In [38]:
couvres_feu.head(1)

,JOUR,LIBELLE_COUVRE_FEU
0,17/10/2020,21h-6h


In [39]:
couvres_feu["JOUR"] = couvres_feu["JOUR"].str.slice(start=6, stop=10)+"-"+couvres_feu["JOUR"].str.slice(start=3, stop=5)+"-"+couvres_feu["JOUR"].str.slice(start=0, stop=2)
couvres_feu["JOUR"] = pd.to_datetime(couvres_feu["JOUR"])

In [40]:
df_finale = df_finale.join(couvres_feu.set_index('JOUR'), on = "JOUR")

In [41]:
df_finale.head()

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE,LIBELLE_VACANCE,LIBELLE_CONFINEMENT,LIBELLE_COUVRE_FEU
18,2015-01-01,ARGENTINE,874,Jour de l’an,NaN,NaN,Noël,NaN,NaN
39,2015-01-01,BASTILLE,5962,Jour de l’an,NaN,NaN,Noël,NaN,NaN
45,2015-01-01,BERAULT,1136,Jour de l’an,NaN,NaN,Noël,NaN,NaN
110,2015-01-01,CHAMPS-ELYSEES-CLEMENCEAU,2994,Jour de l’an,NaN,NaN,Noël,NaN,NaN
115,2015-01-01,CHARLES DE GAULLE ETOILE,10962,Jour de l’an,NaN,NaN,Noël,NaN,NaN


<h3><font color=darkred>Base Finale</h3>

Voici un apperçu de notre **dataset final** :

In [42]:
df_finale.tail()

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE,LIBELLE_VACANCE,LIBELLE_CONFINEMENT,LIBELLE_COUVRE_FEU
1802375,2021-12-31,PORTE MAILLOT,3897,NaN,NaN,NaN,Noël,NaN,NaN
1802394,2021-12-31,REUILLY-DIDEROT,6777,NaN,NaN,NaN,Noël,NaN,NaN
1802429,2021-12-31,SAINT-MANDE-TOURELLE,5087,NaN,NaN,NaN,Noël,NaN,NaN
1802442,2021-12-31,SAINT-PAUL (LE MARAIS),5061,NaN,NaN,NaN,Noël,NaN,NaN
1802495,2021-12-31,TUILERIES,361,NaN,NaN,NaN,Noël,NaN,NaN


In [43]:
df_finale.to_csv("Bases/Base_Finale.csv")

<h3><font color=darkred>Création de bases annexes utiles pour l'analyse déscriptive des données</h3>

Dans cette partie, nous allons créer différents datasets à partir de notre dataset final afin de répondre aux besoins de la partie *analyse descriptive* de notre projet.<br>
<br>
En effet, pour étudier la fréquentation de la ligne de métro 1, nous avons besoin de regrouper le nombre de validations en fonction par exemple du jour de la semaine ou du mois.<br>
<br>
Ces données seront donc disponibles dans les fichiers : 
- Base_Type_jours.csv
- Base_Type_mois.csv

In [44]:
df_finale = pd.read_csv("Bases/Base_Finale.csv", sep=",").drop("Unnamed: 0", axis=1)

In [45]:
df_finale.head(2)

,JOUR,LIBELLE_ARRET,NB_VALD,LIBELLE_JOUR_FERIE,MODALITE_GREVE,LIBELLE_FETE,LIBELLE_VACANCE,LIBELLE_CONFINEMENT,LIBELLE_COUVRE_FEU
0,2015-01-01,ARGENTINE,874,Jour de l’an,NaN,NaN,Noël,NaN,NaN
1,2015-01-01,BASTILLE,5962,Jour de l’an,NaN,NaN,Noël,NaN,NaN


In [46]:
df_finale = df_finale.fillna("Aucun")

In [47]:
df_NBVALD = df_finale.groupby(['JOUR', 'LIBELLE_JOUR_FERIE', 
                               'LIBELLE_FETE', 'MODALITE_GREVE', 'LIBELLE_VACANCE', 
                               'LIBELLE_CONFINEMENT', 'LIBELLE_COUVRE_FEU'])['NB_VALD'].agg('sum')
df_NBVALD.to_csv("Bases/Base_NBVALD.csv")

In [48]:
df_NBVALD = pd.read_csv("Bases/Base_NBVALD.csv", sep=",")

liste_jours = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
liste_mois = ["Janvier", "Février", "Mars", "Avril", "Mai", "Juin", 
              "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"]

df_NBVALD["NUM_JOUR"] = pd.to_datetime(df_NBVALD["JOUR"]).dt.dayofweek
df_NBVALD["TYPO_JOUR"] = df_NBVALD["NUM_JOUR"].apply(lambda row: liste_jours[row])
df_NBVALD["NUM_MOIS"] = pd.to_datetime(df_NBVALD["JOUR"]).dt.month
df_NBVALD["TYPO_MOIS"] = df_NBVALD["NUM_MOIS"].apply(lambda row: liste_mois[row-1])
df_NBVALD["ANNEE"] = pd.to_datetime(df_NBVALD["JOUR"]).dt.year

df_NBVALD.rename(columns = {'JOUR': 'date',
                           'LIBELLE_JOUR_FERIE': 'ferie',
                           'LIBELLE_FETE': 'fete',
                           'MODALITE_GREVE': 'greve',
                           'LIBELLE_VACANCE': 'vacance',
                           'LIBELLE_CONFINEMENT': 'confinement',
                           'LIBELLE_COUVRE_FEU': 'couvre_feu',
                           'NB_VALD': 'nb_validation',
                           'TYPO_JOUR': 'type_jour',
                           'NUM_JOUR': 'num_jour',
                           'TYPO_MOIS': 'type_mois',
                           'NUM_MOIS': 'num_mois',
                           'ANNEE': 'annee',}, inplace = True)

df_NBVALD.to_csv("Bases/Base_NBVALD.csv", index=False)

In [49]:
df_NBVALD = pd.read_csv("Bases/Base_NBVALD.csv", sep=",")

In [50]:
df_NBVALD.head(1)

,date,ferie,fete,greve,vacance,confinement,couvre_feu,nb_validation,num_jour,type_jour,num_mois,type_mois,annee
0,2015-01-01,Jour de l’an,Aucun,Aucun,Noël,Aucun,Aucun,124174,3,Jeudi,1,Janvier,2015


In [51]:
df_typo_jours = df_NBVALD.groupby(["annee", "num_jour", "type_jour"])["nb_validation"].agg('sum')
df_typo_jours.to_csv("Bases/Base_Type_jours.csv")

In [52]:
df_typo_jours.head()

annee  num_jour  type_jour
2015   0         Lundi        29767339
       1         Mardi        33268510
       2         Mercredi     33077000
       3         Jeudi        33208539
       4         Vendredi     32432610
Name: nb_validation, dtype: int64

In [53]:
df_typo_mois = df_NBVALD.groupby(["annee", "num_mois", "type_mois"])["nb_validation"].agg('sum')
df_typo_mois.to_csv("Bases/Base_Type_mois.csv")

In [54]:
df_typo_mois.head()

annee  num_mois  type_mois
2015   1         Janvier      16392408
       2         Février      15030822
       3         Mars         16448609
       4         Avril        16176654
       5         Mai          14746291
Name: nb_validation, dtype: int64

*Maintenant que nous avons fait ce travail de préparation de données, on peut s'attaquer au projet !*

In [55]:
print("time execution :", time.time() - start_time)

time execution : 53.36846327781677


<h2><font color=darkred><center>FIN</center></h2>